# Exercise 5 - Spark in Scala _[4 points]_

In this exercise you have to solve the tasks given below. 


## a) Elementary RDD functions 

(Brushing up on the basics of functional programming Scala)




####  You are given a list of the first 20 numbers of the Fibbonacci numbers. 

In [1]:
val fibs20 = sc.parallelize(List( 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181))  

Intitializing Scala interpreter ...

Spark Web UI available at http://c100.local:8088/proxy/application_1587827373944_0715
SparkContext available as 'sc' (version = 2.4.0-cdh6.3.2, master = yarn, app id = application_1587827373944_0715)
SparkSession available as 'spark'


fibs20: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:25


####  Produce a list that contains all multiples of 3 from the list 'fibs20':

In [2]:
fibs20.filter(x => x % 3 == 0 && x > 0).collect()

res0: Array[Int] = Array(3, 21, 144, 987)


####  Compute the median value of the list 'fibs20':

In [3]:
def median(values: org.apache.spark.rdd.RDD[Int]) = {
    val length = values.count()
    val medianIndex = (math ceil length / 2.0).toInt
    if (length % 2 == 1) {
        values.collect()(medianIndex)
    } else {
        val middles = values.collect().slice(medianIndex, medianIndex + 2)
        (middles(0) + middles(1)) / 2.0
    }
}

median(fibs20)

median: (values: org.apache.spark.rdd.RDD[Int])Double
res1: Double = 72.0


####  Produce a for the list 'fibs20' its standard deviation:

In [4]:
def stdev(values: org.apache.spark.rdd.RDD[Int]) = {
    val length = values.count()
    val sum = values.reduce((x,y) => x + y)
    val avg = sum.toDouble / length
    val diffs = values.map(x => math.pow((x - avg), 2))
    val diffsum = diffs.reduce((x,y) => x + y)
    math sqrt (diffsum / length)
}

stdev(fibs20)

stdev: (values: org.apache.spark.rdd.RDD[Int])Double
res2: Double = 1055.812808929689


####  You are given a random list of words

In [5]:
val words = sc.parallelize(List("pestilence", "virion", "quarantine", "vaccine"))

words: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[3] at parallelize at <console>:25


####  Furthermore, we define a function that maps a word to its list of permutations

In [6]:
def permutate (word:String) = word.permutations.toList

permutate: (word: String)List[String]


####  Produce a single list containing all permutations of elements from the list 'words':

In [7]:
words.flatMap(permutate).collect()

res3: Array[String] = Array(peeestilnc, peeestilcn, peeestinlc, peeestincl, peeesticln, peeesticnl, peeestlinc, peeestlicn, peeestlnic, peeestlnci, peeestlcin, peeestlcni, peeestnilc, peeestnicl, peeestnlic, peeestnlci, peeestncil, peeestncli, peeestciln, peeestcinl, peeestclin, peeestclni, peeestcnil, peeestcnli, peeesitlnc, peeesitlcn, peeesitnlc, peeesitncl, peeesitcln, peeesitcnl, peeesiltnc, peeesiltcn, peeesilntc, peeesilnct, peeesilctn, peeesilcnt, peeesintlc, peeesintcl, peeesinltc, peeesinlct, peeesinctl, peeesinclt, peeesictln, peeesictnl, peeesicltn, peeesiclnt, peeesicntl, peeesicnlt, peeesltinc, peeeslticn, peeesltnic, peeesltnci, peeesltcin, peeesltcni, peeeslitnc, peeeslitcn, peeeslintc, peeeslinct, peeeslictn, peeeslicnt, peeeslntic, peeeslntci, peeeslnitc, peeeslnict, p...

## b) From SQL to Dataframe (and back again)

#### Find for each of the Spark SQL queries an equivalent one that only uses the Dataframe API (or vice versa)


In [8]:
val octDF = spark.read.options(Map("header"->"true")).format("csv").load("/home/adbs20/shared/ecommerce/2019-Oct.csv")
val novDF = spark.read.options(Map("header"->"true")).format("csv").load("/home/adbs20/shared/ecommerce/2019-Nov.csv")
octDF.createOrReplaceTempView("oct")
novDF.createOrReplaceTempView("nov")
// Create RDD view into dataset
val octRDD = octDF.rdd
val novRDD = novDF.rdd

octDF: org.apache.spark.sql.DataFrame = [event_time: string, event_type: string ... 7 more fields]
novDF: org.apache.spark.sql.DataFrame = [event_time: string, event_type: string ... 7 more fields]
octRDD: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[26] at rdd at <console>:29
novRDD: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[30] at rdd at <console>:30


In [9]:
octDF.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



#### Query 1: Transform the given Spark SQL query into the Dataframe API

In [10]:
val query1 = spark.sql("SELECT COUNT(*) FROM oct WHERE brand='samsung'")
query1.show() // 'false' turns of truncation of row entries
query1.explain()

+--------+
|count(1)|
+--------+
| 5282775|
+--------+

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[count(1)])
+- Exchange SinglePartition
   +- *(1) HashAggregate(keys=[], functions=[partial_count(1)])
      +- *(1) Project
         +- *(1) Filter (isnotnull(brand#15) && (brand#15 = samsung))
            +- *(1) FileScan csv [brand#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://nameservice1/home/adbs20/shared/ecommerce/2019-Oct.csv], PartitionFilters: [], PushedFilters: [IsNotNull(brand), EqualTo(brand,samsung)], ReadSchema: struct<brand:string>


query1: org.apache.spark.sql.DataFrame = [count(1): bigint]


In [11]:
val query11 = octDF.where($"brand" === "samsung").agg(count("*"))
query11.show()
query11.explain()

+--------+
|count(1)|
+--------+
| 5282775|
+--------+

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[count(1)])
+- Exchange SinglePartition
   +- *(1) HashAggregate(keys=[], functions=[partial_count(1)])
      +- *(1) Project
         +- *(1) Filter (isnotnull(brand#15) && (brand#15 = samsung))
            +- *(1) FileScan csv [brand#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://nameservice1/home/adbs20/shared/ecommerce/2019-Oct.csv], PartitionFilters: [], PushedFilters: [IsNotNull(brand), EqualTo(brand,samsung)], ReadSchema: struct<brand:string>


query11: org.apache.spark.sql.DataFrame = [count(1): bigint]


#### Query 2: Transform the given Dataframe API query into Spark SQL

In [12]:
val query2 = novDF.groupBy("brand").count()
query2.show(false) // 'false' turns of truncation of row entries
query2.explain()

+---------------------+------+
|brand                |count |
+---------------------+------+
|houseofseasons       |1318  |
|edifier              |3565  |
|yokohama             |128697|
|tuffoni              |3617  |
|tmnt                 |574   |
|serebro              |7105  |
|welss                |4929  |
|tega                 |2317  |
|vortex               |55    |
|dvin                 |76    |
|blaster              |122   |
|crest                |325   |
|receptybabuskiagafi  |22    |
|lykan                |20    |
|muimui               |5     |
|riminibosco          |5     |
|maximumpowernutrition|2     |
|keenway              |2368  |
|rosato               |698   |
|marley               |1063  |
+---------------------+------+
only showing top 20 rows

== Physical Plan ==
*(2) HashAggregate(keys=[brand#43], functions=[count(1)])
+- Exchange hashpartitioning(brand#43, 200)
   +- *(1) HashAggregate(keys=[brand#43], functions=[partial_count(1)])
      +- *(1) FileScan csv [brand#43

query2: org.apache.spark.sql.DataFrame = [brand: string, count: bigint]


In [13]:
val query22 = spark.sql("SELECT brand, COUNT(*) FROM nov GROUP BY brand")
query22.show(false)
query22.explain()

+---------------------+--------+
|brand                |count(1)|
+---------------------+--------+
|yokohama             |128697  |
|welss                |4929    |
|serebro              |7105    |
|edifier              |3565    |
|tega                 |2317    |
|tuffoni              |3617    |
|crest                |325     |
|tmnt                 |574     |
|vortex               |55      |
|houseofseasons       |1318    |
|dvin                 |76      |
|muimui               |5       |
|blaster              |122     |
|riminibosco          |5       |
|receptybabuskiagafi  |22      |
|maximumpowernutrition|2       |
|lykan                |20      |
|sonel                |11501   |
|nutricia             |2496    |
|keenway              |2368    |
+---------------------+--------+
only showing top 20 rows

== Physical Plan ==
*(2) HashAggregate(keys=[brand#43], functions=[count(1)])
+- Exchange hashpartitioning(brand#43, 200)
   +- *(1) HashAggregate(keys=[brand#43], functions=[partial

query22: org.apache.spark.sql.DataFrame = [brand: string, count(1): bigint]


#### Query 3: Transform the given Spark SQL query into the Dataframe API

In [14]:
val query3 = spark.sql("SELECT user_id, brand from oct where event_type ='purchase' AND brand IS NOT NULL GROUP BY user_id,brand HAVING COUNT(*) >= (SELECT COUNT(brand) from oct as oct2 where event_type ='purchase' AND oct2.user_id = oct.user_id GROUP BY user_id)  ")
query3.show(false) // 'false' turns of truncation of row entries
query3.explain()

+---------+--------+
|user_id  |brand   |
+---------+--------+
|434170823|lucente |
|441522689|oppo    |
|461023190|fender  |
|470193237|huawei  |
|490354170|viatti  |
|512369688|topface |
|512388342|acer    |
|512389344|samsung |
|512392219|berghoff|
|512395692|xiaomi  |
|512397473|apple   |
|512405720|jbl     |
|512412345|indesit |
|512420187|philips |
|512420588|apple   |
|512425298|xiaomi  |
|512429071|dongma  |
|512432852|apple   |
|512442599|xiaomi  |
|512444057|apple   |
+---------+--------+
only showing top 20 rows

== Physical Plan ==
*(7) Project [user_id#17, brand#15]
+- *(7) Filter (count(1)#188L >= if (isnull(alwaysTrue#191)) 0 else count(brand)#186L)
   +- SortMergeJoin [user_id#17], [user_id#17#190], LeftOuter
      :- *(3) Sort [user_id#17 ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(user_id#17, 200)
      :     +- *(2) HashAggregate(keys=[user_id#17, brand#15], functions=[count(1)])
      :        +- Exchange hashpartitioning(user_id#17, brand#15, 2

query3: org.apache.spark.sql.DataFrame = [user_id: string, brand: string]


In [15]:
val subquery32 = octDF.where("event_type ='purchase'").groupBy("user_id").agg(count("brand").alias("count2"))
val query32 = octDF.select($"user_id".alias("user_id_1"), $"brand").where("event_type = 'purchase' and brand is not null")
.groupBy("user_id_1", "brand").agg(count("*").alias("count"))
.join(subquery32, $"user_id_1" === $"user_id")
.filter($"count" >= $"count2")
.select("user_id", "brand")
query32.show(false)
query32.explain()

+---------+--------+
|user_id  |brand   |
+---------+--------+
|434170823|lucente |
|441522689|oppo    |
|461023190|fender  |
|470193237|huawei  |
|490354170|viatti  |
|512369688|topface |
|512388342|acer    |
|512389344|samsung |
|512392219|berghoff|
|512395692|xiaomi  |
|512397473|apple   |
|512405720|jbl     |
|512412345|indesit |
|512420187|philips |
|512420588|apple   |
|512425298|xiaomi  |
|512429071|dongma  |
|512432852|apple   |
|512442599|xiaomi  |
|512444057|apple   |
+---------+--------+
only showing top 20 rows

== Physical Plan ==
*(6) Project [user_id#17, brand#15]
+- *(6) SortMergeJoin [user_id_1#210], [user_id#17], Inner, (count#217L >= count2#207L)
   :- *(3) Sort [user_id_1#210 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(user_id_1#210, 200)
   :     +- *(2) HashAggregate(keys=[user_id_1#210, brand#15], functions=[count(1)])
   :        +- Exchange hashpartitioning(user_id_1#210, brand#15, 200)
   :           +- *(1) HashAggregate(keys=[user_id_1#210,

subquery32: org.apache.spark.sql.DataFrame = [user_id: string, count2: bigint]
query32: org.apache.spark.sql.DataFrame = [user_id: string, brand: string]


#### Query 4: Transform the given Dataframe API query into Spark SQL

In [16]:
val query4 = octDF
      .join(novDF, octDF("user_id") === novDF("user_id"))
      .filter(octDF("event_type") === "view" )
      .groupBy(octDF("category_id"))
      .agg(avg(novDF("price")).as("avgPrice"))
      .orderBy(desc("avgPrice"))
query4.show(false) // 'false' turns of truncation of row entries
query4.explain()

+-------------------+------------------+
|category_id        |avgPrice          |
+-------------------+------------------+
|2053013557544485393|563.5393907361115 |
|2053013560497275585|531.4014232236495 |
|2053013558920217191|452.44963834043574|
|2053013566503518219|432.4151428571424 |
|2134905019189691101|431.7818269851515 |
|2160912607672795565|420.6898284273414 |
|2053013560555995845|415.17758445401313|
|2053013561092866779|408.38360056087237|
|2053013560463721151|392.2455597355071 |
|2053013561059312345|382.69055976121973|
|2116907525201723525|375.40895114020424|
|2053013564523807647|374.87466637575403|
|2160912655915679819|372.72334449178317|
|2053013552695869677|372.65947867882153|
|2053013555724157349|370.69754656974106|
|2053013557645148695|363.24151916201055|
|2105319817211805903|355.39508981216244|
|2053013560530830019|355.10748622163754|
|2109094148110811752|350.7298993914582 |
|2053013559406756479|347.27060201997256|
+-------------------+------------------+
only showing top

query4: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [category_id: string, avgPrice: double]


In [17]:
val query42 = spark.sql("SELECT o.category_id, AVG(n.price) as avgPrice FROM oct o, nov n WHERE o.event_type = 'view' AND o.user_id = n.user_id GROUP BY o.category_id ORDER BY avgPrice DESC")
query42.show(false)
query42.explain()

+-------------------+------------------+
|category_id        |avgPrice          |
+-------------------+------------------+
|2053013557544485393|563.5393907361117 |
|2053013560497275585|531.4014232236497 |
|2053013558920217191|452.4496383404357 |
|2053013566503518219|432.41514285714237|
|2134905019189691101|431.7818269851517 |
|2160912607672795565|420.6898284273414 |
|2053013560555995845|415.17758445401296|
|2053013561092866779|408.3836005608722 |
|2053013560463721151|392.2455597355071 |
|2053013561059312345|382.6905597612197 |
|2116907525201723525|375.4089511402043 |
|2053013564523807647|374.874666375754  |
|2160912655915679819|372.72334449178317|
|2053013552695869677|372.65947867882153|
|2053013555724157349|370.69754656974135|
|2053013557645148695|363.2415191620105 |
|2105319817211805903|355.3950898121623 |
|2053013560530830019|355.1074862216376 |
|2109094148110811752|350.7298993914584 |
|2053013559406756479|347.27060201997267|
+-------------------+------------------+
only showing top

query42: org.apache.spark.sql.DataFrame = [category_id: string, avgPrice: double]


### For All Queries Above: 
#### Analyze the plans (.explain() ) and compare performance (using the Internal Spark Web UI). Try to reason about any major differences in the logical plans (if there are any).

## c) Wide and Narrow Dependencies

#### Look at the Dataframe queries given as part of b) or for which you wrote the Dataframe version.

#### Use the Internal Spark Web UI to analyse the dependencies and stages of the queries, and try to determine which commands on which Dataframes are executed as wide dependencies and which as narrow dependencies. 
